In [1]:
import quantfreedom as qf
import numpy as np
import pandas as pd

from quantfreedom.backtester.indicators.talib_ind import from_talib
from quantfreedom._typing import (
    pdFrame, pdSeries
)
prices = pd.read_csv('30min.csv', index_col='time')
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 20],
)


In [32]:
def eval_is_below(
    data_testing,
    thing,
):
    if not isinstance(data_testing, pdFrame):
        raise ValueError(
            "Data must be a dataframe with multindex")

    data_values = data_testing.values
    eval_array = np.empty_like(data_values, dtype=np.bool_)
    final_df = pd.DataFrame()
    ind_name_below = data_testing.columns.names[0].split('_')[
        0] + '_is_below'

    if isinstance(thing, list):
        if not all(isinstance(x, (int, float)) for x in thing):
            raise ValueError(
                "Thing must be a list of ints or floats")
        for col in range(data_testing.shape[1]):
            temp_data_array = data_values[:, col]
            for x in range(len(thing)):
                for bar in range(data_values.shape[0]):
                    eval_array[bar, col] = np.where(
                        temp_data_array[bar] < thing[x], True, False)

                temp_df = pd.DataFrame(
                    eval_array[:, col],
                    index=data_testing.index,
                    columns=pd.MultiIndex.from_tuples(
                        [data_testing.columns[col] + (thing[x],)],
                        names=list(data_testing.columns.names) +
                        [ind_name_below]
                    )
                )
                final_df = pd.concat([final_df, temp_df], axis=1)

    elif isinstance(thing, (int, float)):
        for col in range(data_testing.shape[1]):
            temp_data_array = data_values[:, col]
            for bar in range(data_values.shape[0]):
                eval_array[bar, col] = np.where(
                    temp_data_array[bar] < thing, True, False)
            temp_df = pd.DataFrame(
                eval_array[:, col],
                index=data_testing.index,
                columns=pd.MultiIndex.from_tuples(
                    [data_testing.columns[col] + (thing,)],
                    names=list(data_testing.columns.names) + [ind_name_below]
                )
            )
            final_df = pd.concat([final_df, temp_df], axis=1)
    else:
        raise ValueError(
            "Thing must be a list of ints or floats or int or float ")

    return final_df


eval_is_below(
    rsi_ind,
    [20, 25, 30, 40, 50]
).head(40)


RSI_count                0                                  1                \
timeperiod              15                                 20                 
RSI_is_below            20     25     30     40     50     20     25     30   
time                                                                          
2020-10-01 00:00:00  False  False  False  False  False  False  False  False   
2020-10-01 00:30:00  False  False  False  False  False  False  False  False   
2020-10-01 01:00:00  False  False  False  False  False  False  False  False   
2020-10-01 01:30:00  False  False  False  False  False  False  False  False   
2020-10-01 02:00:00  False  False  False  False  False  False  False  False   
2020-10-01 02:30:00  False  False  False  False  False  False  False  False   
2020-10-01 03:00:00  False  False  False  False  False  False  False  False   
2020-10-01 03:30:00  False  False  False  False  False  False  False  False   
2020-10-01 04:00:00  False  False  False  False  False  False  False  False   
2020-10-01 04:30:00  False  False  False  False  False  False  False  False   
2020-10-01 05:00:00  False  False  False  False  False  False  False  False   
2020-10-01 05:30:00  False  False  False  False  False  False  False  False   
2020-10-01 06:00:00  False  False  False  False  False  False  False  False   
2020-10-01 06:30:00  False  False  False  False  False  False  False  False   
2020-10-01 07:00:00  False  False  False  False  False  False  False  False   
2020-10-01 07:30:00  False  False  False  False  False  False  False  False   
2020-10-01 08:00:00  False  False  False  False  False  False  False  False   
2020-10-01 08:30:00  False  False  False  False  False  False  False  False   
2020-10-01 09:00:00  False  False  False  False  False  False  False  False   
2020-10-01 09:30:00  False  False  False  False  False  False  False  False   
2020-10-01 10:00:00  False  False  False  False  False  False  False  False   
2020-10-01 10:30:00  False  False  False  False  False  False  False  False   
2020-10-01 11:00:00  False  False  False  False  False  False  False  False   
2020-10-01 11:30:00  False  False  False  False  False  False  False  False   
2020-10-01 12:00:00  False  False  False  False  False  False  False  False   
2020-10-01 12:30:00  False  False  False  False  False  False  False  False   
2020-10-01 13:00:00  False  False  False  False  False  False  False  False   
2020-10-01 13:30:00  False  False  False  False  False  False  False  False   
2020-10-01 14:00:00  False  False  False  False  False  False  False  False   
2020-10-01 14:30:00  False  False  False  False  False  False  False  False   
2020-10-01 15:00:00  False  False  False  False  False  False  False  False   
2020-10-01 15:30:00  False  False  False   True   True  False  False  False   
2020-10-01 16:00:00  False   True   True   True   True  False  False   True   
2020-10-01 16:30:00   True   True   True   True   True  False   True   True   
2020-10-01 17:00:00   True   True   True   True   True  False   True   True   
2020-10-01 17:30:00   True   True   True   True   True  False   True   True   
2020-10-01 18:00:00  False  False   True   True   True  False  False  False   
2020-10-01 18:30:00  False  False  False   True   True  False  False  False   
2020-10-01 19:00:00  False  False  False   True   True  False  False  False   
2020-10-01 19:30:00  False  False  False   True   True  False  False  False   

RSI_count                          
timeperiod                         
RSI_is_below            40     50  
time                               
2020-10-01 00:00:00  False  False  
2020-10-01 00:30:00  False  False  
2020-10-01 01:00:00  False  False  
2020-10-01 01:30:00  False  False  
2020-10-01 02:00:00  False  False  
2020-10-01 02:30:00  False  False  
2020-10-01 03:00:00  False  False  
2020-10-01 03:30:00  False  False  
2020-10-01 04:00:00  False  False  
2020-10-01 04:30:00  False  False  
2020-10-01 05:00:00

In [3]:
ind_settings_tup = (0, 20)
param_keys = ['rsi_count', 'timeperiod']
pd_index = pd.MultiIndex.from_tuples(
    [ind_settings_tup],
    names=param_keys
)


In [4]:
list(pd_index.names)


['rsi_count', 'timeperiod']

In [5]:
ind_name_below = param_keys[0].split('_')[0] + '_is_below'
pd.MultiIndex.from_tuples(
    [pd_index[0] + (30,)],
    names=list(pd_index.names) + [ind_name_below]
)


MultiIndex([(0, 20, 30)],
           names=['rsi_count', 'timeperiod', 'rsi_is_below'])

In [6]:
qf.clear_cache()
